In [ ]:
#!pip install panel
#!pip install holoviews
import panel as pn
import param
import holoviews as hv,panel as pn,param
import holoviews as hv
from holoviews import streams
hv.extension('bokeh')
import numpy as np


In [ ]:
def Prox(z,h,p):
    eps=0.00001
    # Calcule le prox de la focntion h*F où F(x)=x^p au point z pour p>1
    x=z
    for k in np.arange(3):
        x=x-(h*p*np.sign(x)*(np.abs(x))**(p-1)+x-z)/(h*p*(p-1)*np.sign(x)*(np.abs(x))**(p-2)+1)
    f=h*p*np.sign(x)*(np.abs(x))**(p-1)+x-z
    return x,f


In [ ]:
x,f=Prox(2,0.01,1.5)
print(x,f)

In [ ]:
def Nesterov(p,h,alpha,Nb,x0):
    x=np.zeros(Nb)
    x[0]=x0
    x[1]=x0-h*p*(x[0])**(p-1)
    y=np.copy(x)
    for k in range(1,Nb-1):
        y[k+1]=x[k]+(k/(k+alpha))*(x[k]-x[k-1])
        if p<2:
            x[k+1],f=Prox(y[k+1],h,p)
        else:
            x[k+1]=y[k+1]-h*p*np.sign(y[k+1])*(np.abs(y[k+1])**(p-1))
    return x

In [ ]:
def rate(alpha,p):
    if p<=2:
        return 2*alpha*p/(p+2)
    if p>2:
        return min(2*alpha*p/(p+2),2*p/(p-2))

In [ ]:
PMAX=8
ALPHAMAX=7
p= np.arange(1, PMAX,0.01)
alpha = np.arange(0, ALPHAMAX, 0.01)
A,P= np.meshgrid(alpha,p)
vrate=np.vectorize(rate)
R=vrate(A,P)
Rate = hv.Dataset((alpha, p,R),['alpha', 'p'], 'rate')
RateVisu=Rate.to(hv.Image, ['alpha', 'p'])
C1=RateVisu.sample(p=5)
C2=RateVisu.sample(p=2)
C3=RateVisu.sample(p=1.5)
C4=RateVisu.sample(p=3)
hv.Curve(C2).opts(color='r',width=600,line_dash='dotted')\
*hv.Curve(C3).opts(color='g',line_dash='dashed')\
*hv.Curve(C4).opts(color='c',line_dash='dashdot')\
*hv.Curve(C1).opts(color='k',line_dash='solid').redim.range(alpha=(0, ALPHAMAX),rate=(0, 8.0))

In [ ]:
def nesterov(alpha,p,x0=1,step=0.0000005,T = 70):
    Nb=int(np.floor(T/np.sqrt(step)))
    xnest=Nesterov(p,step,alpha,Nb,x0)
    sousech = 10
    xrescale=np.abs(xnest[0::sousech])**p*np.arange(0,Nb,sousech)**rate(alpha,p)
    xrescale = xrescale/np.max(xrescale)
    n1=len(xrescale)
    time=np.linspace(0,T,n1)
    pt="%2.2f" % rate(alpha,p)
    stral=str(alpha)
    strp=str(p)
    te1='alpha='+stral+', gamma='+strp
    formula='$\alpha$'
    te2='Expected rate : '+pt
    #TN=pn.Column(LaTeX(te1,size=25,dpi=100),LaTeX(te2,size=25,dpi=100))
    ht1=hv.Text(1/2, 1/3, 'Expected rate : '+pt).opts(text_font_size='18pt').opts(xaxis=None,yaxis=None)
    ht2=hv.Text(1/2, 2/3, te1).opts(text_font_size='18pt').opts(xaxis=None,yaxis=None)
    ht=ht1*ht2.opts(toolbar=None)
    h2 = hv.Curve((time,xrescale),kdims='time',vdims = 'rescale').options(width=600)
    return pn.Row(h2,ht)

In [ ]:
p=3
alpha=1
nesterov(alpha,p,x0=1,step=0.00000001,T = 20)


In [ ]:
p=3
alpha=4
nesterov(alpha,p,x0=1,step=0.0000005,T = 1000)

In [ ]:
p=3
alpha=6
nesterov(alpha,p,x0=1,step=0.0000005,T = 200)

In [ ]:
p=3
alpha=8
nesterov(alpha,p,x0=1,step=0.0000001,T = 200)

In [ ]:
p=2
alpha=1
nesterov(alpha,p,x0=1,step=0.00000001,T = 15)

In [ ]:
p=2
alpha=6
nesterov(alpha,p,x0=1,step=0.00000001,T = 15)

In [ ]:
p=1.5
alpha=6
nesterov(alpha,p,x0=1,step=0.00000001,T = 15)

In [ ]:
p=1.5
alpha=1
nesterov(alpha,p,x0=1,step=0.00000001,T = 15)